In [ ]:
np.random.seed(2971)

N = 100  # 100 lines
a = stats.norm.rvs(loc=178, scale=20, size=N)
b = stats.norm.rvs(loc=0, scale=10, size=N)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].set_xlim(unit_price.min(), unit_price.max())
ax[0].set_ylim(-100, 400)
ax[0].set_xlabel('unit_price')
ax[0].set_ylabel('mrt_distance')
#ax[0].axhline(0, c="k", ls="--")
#ax[0].axhline(272, c="k")
ax[0].set_title("b ~ Normal(0,10²)")
xbar = unit_price.mean()
x = np.array([unit_price.min(), unit_price.max()])
for i in range(N):
    ax[0].plot(x, a[i] + b[i] * (x - xbar), "k", alpha=0.2)

b = stats.lognorm.rvs(s=1, scale=1, size=100)

ax[1].set_xlim(unit_price.min(), unit_price.max())
ax[1].set_ylim(-100, 400)
ax[1].set_xlabel('unit_price')
ax[0].set_ylabel('mrt_distance')
#ax[1].axhline(0, c="k", ls="--", label="embryo")
#ax[1].axhline(272, c="k")
ax[1].set_title("log(b) ~ Normal(0,10²)")
#ax[1].text(x=35, y=282, s="World's tallest person (272cm)")
#ax[1].text(x=35, y=-25, s="Embryo");
for i in range(N):
    ax[1].plot(x, a[i] + b[i] * (x - xbar), "k", alpha=0.2)

In [ ]:
with pm.Model() as m4_3:
    a = pm.Normal("a", mu=178, sigma=20)
    b = pm.Lognormal("b", mu=0, sigma=1)
    sigma = pm.Uniform("sigma", 0, 50)
    mu = a + b * (unit_price.values - xbar)
    mrt = pm.Normal('mrt_distance', mu=mu, sigma=sigma, observed=mrt_distance.values)
    trace_4_3 = pm.sample(1000, tune=1000)

In [ ]:
# Model with the log(b) ~ Normal(0, 10) prior
with pm.Model() as m4_3b:
    a = pm.Normal("a", mu=178, sigma=20)
    log_b = pm.Normal("log_b", mu=0, sigma=1)
    sigma = pm.Uniform("sigma", 0, 50)
    mu = a + np.exp(log_b) * (unit_price.values - xbar)
    mrt = pm.Normal('mrt_distance', mu=mu, sigma=sigma, observed=real_estate['X3 distance to the nearest MRT station'])
    trace_4_3b = pm.sample(1000, tune=1000)

In [ ]:
az.summary(trace_4_3, kind="stats")

In [ ]:
trace_4_3_df = trace_4_3.posterior.to_dataframe()
trace_4_3_df.cov().round(3)

In [ ]:
plt.scatter(unit_price, real_estate['X3 distance to the nearest MRT station'], alpha=0.5)
plt.plot(
    unit_price,
    trace_4_3.posterior["a"].values.mean()
        + trace_4_3.posterior["b"].values.mean() * (mrt_distance - xbar),
    color='C1')
plt.xlabel('unit price')
plt.ylabel('mrt distance');

In [ ]:
trace_4_3_df.head(5)